<a href="https://colab.research.google.com/github/Muhammadsulton1/MIPT_Data_analys/blob/main/%D0%BF%D0%BE%D0%B8%D1%81%D0%BA_%D0%B0%D0%BD%D0%BE%D0%BC%D0%B0%D0%BB%D0%B8%D0%B9_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')  # стиль для графиков
%matplotlib inline

Зачем вообще искать аномалии?
Предобработка данных: убираем выбросы, чтобы не переобучиться
Анализ выбросов: хотим найти, понять и обезвредить
Поиск аномалий как самоцель (поиск мошенников, подозрительного поведения пользователей)
Поиск аномалий может быть как конечной целью анализа и построения моделей, так и промежуточным этапом подготовки и очистки данных. В первом сценарии мы хотим научиться для каждого объекта выборки выносить вердикт, является ли он аномальным/нестандартным, а во-втором мы находим и убираем выбросы в данных, чтобы в дальнейшем получить более устойчивые модели.

В определении из документации scikit-learn, здача поиска аномалий разделяется на два возможных типа:

Outlier detection (поиск выбросов): в тренировочной выборке содержатся выбросы, которые определяются как наблюдения, лежащие далеко от остальных. Таким образом, алгоритмы для детектирования выбросов пытаются найти регионы, где сосредоточена основная масса тренировочных данных, игрорируя аномальные наблюдения.
Novelty detection (поиск "новизны"): тренировочная выборка не загрязнена выбросами, и мы хотим научиться отвечать на вопрос "является ли новое наблюдение выбросом".

#Сложности при поиске аномалий¶
На практике задача поиска аномалий зачастую не сводится к построению бинарного классификатора "выброс/не выброс". Реальные данные редко бывают размечены и мы вынуждены использовать методы обучения без учителя.

Одновременно с этим возникает вопрос о построении надежной схемы проверки результатов, ведь если "правильных ответов" у нас нет, то и понять, насколько алгоритм справляется со своей задачей, уже сложнее. Здесь очень помогут экспертные оценки о проценте аномальных объектов, которые ожидаются в выборке, так как с ними можно будет сравнивать прогнозные значения и варьировать тем самым чувствительность алгоритмов.

Если же поиск и очистка от аномалий - это шаг в подготовке данных для последующего моделирования, то можно использовать стандартные подходы из машинного обученияб, о которых речь пойдёт на будующих неделях нашей специализации.

In [ ]:
df = pd.read_csv('memes_prepare.csv', sep='\t')
df.set_index('name', inplace=True)

# возьмём только числовые колонки
df = df[['views', 'photos', 'comments', 'days_from_creation',
          'average_views', 'average_comments', 'tags_len' ]]
df.head()


Описание колонок:

name - название мема

views - число просмотров на сайте

comments - число комментариев

photos - число вкладышей с мемом

days_from_creation - сколько дней прошло от появления мема

average_views - среднее число просмотров за день

average_comments - среднее число комментариев за день

tags_len - длина тегов в описании в числе символов

In [ ]:

df.hist(figsize=(20, 8));

Прологарифмируем все переменные с длинными хвостами, чтобы на визуализациях аномалий можно было их увидеть.

In [ ]:
for var in ['views', 'photos', 'average_views',
            'average_comments', 'comments']:
    df[var] = df[var].apply(lambda w: np.log(w + 1))

In [ ]:
df.hist(figsize=(20, 8));

Мы видим, что даже после логарифмирования часть хвостов остались тяжёлыми. Давайте попробуем найти все аномальные мемы, лежащие в этих хвостах хотябы по одному из признаков.


#2. Методы на основе описательных статистик¶
#2.1 Правило 3-х сигм
Всё, что оказалось за пределами трёх сигм - аномалия.

In [ ]:
def outlier_std(data, col, threshold=3):
    """
        Вычисляет для каждой строки является ли она аномалией
    """

    mean = data[col].mean()
    std = data[col].std()

    up_bound = mean + threshold * std
    low_bound = mean - threshold * std

    anomalies = pd.concat([data[col] > up_bound, data[col] < low_bound], axis=1).any(axis=1)
    return anomalies, up_bound, low_bound

In [ ]:

a,l,r = outlier_std(df, df.columns)
l

In [ ]:
def get_column_outliers(data, function=outlier_std, threshold=3):

    # дополнительная колонка с отметкой является ли конкретное наблюдение аномалией
    outliers = pd.Series(data=[False]*len(data), index=data.index, name='is_outlier')

    # табличка для статистики по каждой колонке
    comparison_table = {}

    for column in data.columns:
        anomalies, upper_bound, lower_bound = function(data, column, threshold=threshold)
        comparison_table[column] = [upper_bound, lower_bound, sum(anomalies), 100*sum(anomalies)/len(anomalies)]
        outliers.loc[anomalies[anomalies].index] = True

    comparison_table = pd.DataFrame(comparison_table).T
    comparison_table.columns=['upper_bound', 'lower_bound', 'anomalies_count', 'anomalies_percentage']
    return comparison_table, outliers

In [ ]:
comparison_table, std_outliers = get_column_outliers(df)

# статистика по каждой колонке и числу аномалий в ней
comparison_table

In [ ]:
# какие наблюдения являются аномалиями, а какие нет
std_outliers

In [ ]:
def anomalies_report(outliers):
    print("Total number of outliers: {}\nPercentage of outliers:   {:.2f}%".format(
            sum(outliers), 100*sum(outliers)/len(outliers)))

In [ ]:
anomalies_report(std_outliers)

In [ ]:
labeled_df = df.copy()
labeled_df['is_outlier'] = std_outliers

In [ ]:
sns.pairplot(data=labeled_df, vars=df.columns,
             hue='is_outlier', hue_order=[1, 0],
             markers=['x', 'o'],  palette='bright');


#2.2 Межквартильное отклонение¶
Всё, что оказалось за пределами трёх межквартильных отклонений - аномалия.

In [ ]:
def outlier_iqr(data, col, threshold = 3):

    # интерквантильный размах
    IQR = data[col].quantile(0.75) - data[col].quantile(0.25)

    # насколько размахов отступать
    up_bound = data[col].quantile(0.75) + (IQR * threshold)
    low_bound = data[col].quantile(0.25) - (IQR * threshold)

    anomalies = pd.concat([data[col]>up_bound, data[col]<low_bound], axis=1).any(axis=1)
    return anomalies, up_bound, low_bound

In [ ]:
comparison_table, iqr_outliers = get_column_outliers(df, function=outlier_iqr)
anomalies_report(iqr_outliers)

In [ ]:
comparison_table

In [ ]:
labeled_df = df.copy()
labeled_df['is_outlier'] = iqr_outliers

sns.pairplot(data=labeled_df, vars = df.columns,
             hue='is_outlier', hue_order=[1, 0],
             markers=['x', 'o'],  palette='bright');

#3. Кластеризация

In [ ]:
from sklearn.base import BaseEstimator
from scipy.spatial.distance import cdist
from sklearn.preprocessing import RobustScaler

class DistanceOutliers(BaseEstimator):
    """
    Distance based outlier detector model

    Fit method calculates centroid of training samples and
    using `metric` get distances from centroid to train samples.
    Having distances - we calculate `threshold` based on `percentile`.

    Predict method uses `threshold` and `metric` to determine, whether
    distance to sample from centroid is too large -> outlier.

    -----------
    Parameters:

    - metric: string, default - euclidean
        metric to use for distance calculation (see scipy.spatial.distance.cdist)

    - percentile: float in range [0, 100]
        hyperparameter which sets the threshold for anomalies
    """
    def __init__(self, metric='euclidean', percentile=90):
        self.metric = metric
        self.percentile = percentile

    def fit(self, X):
        self.centroid = np.mean(X, axis=0).values.reshape(-1, 1).T
        distances_train = cdist(self.centroid, X, metric=self.metric).reshape(-1)
        self.threshold = np.percentile(distances_train, self.percentile)

    def predict(self, X):
        distances = cdist(self.centroid, X, metric=self.metric).reshape(-1)
        predictions = (distances > self.threshold).astype(int)
        return predictions

In [ ]:
scaler = RobustScaler()
scaled_data = pd.DataFrame(
    data=scaler.fit_transform(data_features),
    columns=data_features.columns
)

In [ ]:
from MulticoreTSNE import MulticoreTSNE as TSNE
tsne = TSNE(perplexity=50, n_jobs=-1)
tsne_transformed = tsne.fit_transform(scaled_data)

plt.figure(figsize=(10, 10))
plt.scatter(tsne_transformed[:, 0], tsne_transformed[:, 1]);

In [ ]:
euclidian_model = DistanceOutliers(metric='euclidean', percentile=90)
euclidian_model.fit(scaled_data)
euclidian_outliers = euclidian_model.predict(scaled_data)
anomalies_report(euclidian_outliers)

In [ ]:
labeled_data = data_features.copy()
labeled_data['is_outlier'] = euclidian_outliers

sns.pairplot(data=labeled_data, vars = other_features,
             hue='is_outlier', hue_order=[1, 0],
             markers=['x', 'o'],  palette='bright')

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(tsne_transformed[:, 0], tsne_transformed[:, 1], c=euclidian_outliers);

In [ ]:
citiblock_model = DistanceOutliers(metric='cityblock', percentile=90)
citiblock_model.fit(scaled_data)
cityblock_outliers = citiblock_model.predict(scaled_data)
anomalies_report(cityblock_outliers)

labeled_data = data_features.copy()
labeled_data['is_outlier'] = cityblock_outliers

sns.pairplot(data=labeled_data, vars = other_features,
             hue='is_outlier', hue_order=[1, 0],
             markers=['x', 'o'],  palette='bright');

plt.figure(figsize=(10, 10))
plt.scatter(tsne_transformed[:, 0], tsne_transformed[:, 1], c=cityblock_outliers);

Алгоритм:

Выбираем случайную точку и находим её соседей в заданной окрестности
Если соседей меньше критического значения – называем выбросами
Если нет – объединяем в «плотный» кластер и повторяем поиск соседей
Если все плотные точки пройдены и помечены как посещенные – выбираем новую не посещенную точку и начинаем сначала
Повторяем, пока все точки не будут посещены

Преимущества:

Density-based (плотностной/вероятностный) метод – умеет в сложные формы кластеров
Поиск выбросов и аномалий в данных
Недостатки:

Довольно сложный в настройке – очень чувствителен к параметру ”плотности” epsilon
Идея - аномалии должны сильно отличаться от основных данных и скорее всего попадут в "шум". Почему бы не увеличивать epsilon до тех пор, пока все "плотные" данные не окажутся в нескольких немногочисленных кластерах, а шума будет столько, сколько мы подозреваем должно быть аномалий

In [ ]:
from sklearn.cluster import DBSCAN

# для начала считаем все наблюдения аномальными
outlier_percentage = 1.

num_clusters = []
anomaly_percentage = []

# берем маленький эпсилон и начинаем увеличивать
eps = 0.05
eps_history = [eps]
while outlier_percentage>0.1:
    model = DBSCAN(eps=eps).fit(scaled_data)
    labels = model.labels_
    num_clusters.append(len(np.unique(labels))-1)
    labels = np.array([1 if label == -1 else 0 for label in labels])
    # считаем текущий процент "шума"
    outlier_percentage = sum(labels==1) / len(labels)
    eps += 0.05
    eps_history.append(eps)
    anomaly_percentage.append(outlier_percentage)

model = DBSCAN(eps)
model.fit(scaled_data)
density_outlier = np.array([1 if label == -1 else 0 for label in model.labels_])

In [ ]:
anomalies_report(density_outlier)

In [ ]:
iterations = eps_history[:-1]

fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('epsilon')
ax1.set_ylabel('number of clusters', color=color)
ax1.plot(iterations, num_clusters, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('anomaly percentage', color=color)  # we already handled the x-label with ax1
ax2.plot(iterations, anomaly_percentage, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
labeled_data = data_features.copy()
labeled_data['is_outlier'] = density_outlier

sns.pairplot(data=labeled_data, vars = other_features,
             hue='is_outlier', hue_order=[1, 0],
             markers=['x', 'o'],  palette='bright');


Коротко о Support Vector Machine
Если совсем вкратце, SVM - базовая линейная модель. Основная идея алгоритма (в случае с классификацией) - разделить классы гиперплоскостью так, чтобы максимизировать расстояние (зазор) между ними. Изначально алгоритм был способен работать только с линейно разделимыми классами, однако в 90-е годы прошлого века метод стал особенно популярен из-за внедрения "Kernel Trick" (1992), позволившего эффективно работать с линейно неразделимыми данными.



Kernel Trick
Ядро (kernel) - это функция, которая способна преобразовать признаковое пространство (в том числе нелинейно), без непосредственного преобразования признаков.

Крайне эффективна в плане вычисления и потенциально позволяет получать бесконечноразмерные признаковые пространства.

Идея заключается в том, что классы, линейно неразделимые в текущем признаковом пространстве, могут стать разделимыми в пространствах более высокой размерности:



One Class SVM
http://rvlasveld.github.io/blog/2013/07/12/introduction-to-one-class-support-vector-machines/

One Class SVM - это одна из форм классического алгоритма, однако, как следует из названия, для его обучения нам достаточно иметь всего один класс, пусть даже и немного "зашумленный", при этом мы хотим научиться для каждого нового наблюдения принимать решение, является ли оно аномальным или нет.

Общая идея: преобразовать признаковое пространство и провести разделяющую гиперплоскость так, чтобы наблюдения лежали как можно дальше от начала координат:



В результате мы получаем границу, по одну сторону которой лежат максимально "плотные" и похожие друг на друга наблюдения из нашей тренировочной выборки, а по другую будут находится аномальные значения, не похожие на все остальные. Процент таких аномальных наблюдений, которые модель будет пытаться отделить от основной части выборки, мы снова задаём в самом начале обучения при помощи параметра nu

In [ ]:
from sklearn.svm import OneClassSVM

one_class_svm = OneClassSVM(nu=0.1, gamma='auto')
one_class_svm.fit(scaled_data)
svm_outliers = one_class_svm.predict(scaled_data)
svm_outliers = np.array([1 if label == -1 else 0 for label in svm_outliers])

In [ ]:
anomalies_report(svm_outliers)

In [ ]:
labeled_data = data_features.copy()
labeled_data['is_outlier'] = svm_outliers

sns.pairplot(data=labeled_data, vars = other_features,
             hue='is_outlier', hue_order=[1, 0],
             markers=['x', 'o'],  palette='bright');


Плюсы и минусы
+ Благодаря kernel trick, модель способна проводить нелинейные разделяющие границы

+ Особенно удобно использовать, когда в данных недостаточно "плохих" наблюдений, чтобы использовать стандартный подход обучения с учителем - бинарную классификацию

- Может очень сильно переобучиться и выдавать большое количество ложно отрицательных результатов, если разделяющий зазор слишком мал

Идея - давайте посмотрим, насколько легко можно "изолировать" наблюдение от всех остальных. Если слишком легко, наверное она лежит далеко и является выбросом. Если очень тяжело - скорее всего она похожа на кучу других точек и выбросом не является.





Алгоритм:

Select the point to isolate.
For each feature, set the range to isolate between the minimum and the maximum.
Choose a feature randomly.
Pick a value that’s in the range, again randomly:
If the chosen value keeps the point above, switch the minimum of the range of the feature to the value.
If the chosen value keeps the point below, switch the maximum of the range of the feature to the value.
Repeat steps 3 & 4 until the point is isolated. That is, until the point is the only one which is inside the range for all features.
Count how many times you’ve had to repeat steps 3 & 4. We call this quantity the isolation number.
https://quantdare.com/isolation-forest-algorithm/

In [ ]:
from sklearn.ensemble import IsolationForest

isolation_forest = IsolationForest(n_estimators=100, contamination=0.1,
                                   max_features=1.0, bootstrap=True, behaviour="new")
isolation_forest.fit(scaled_data)

isolation_outliers = isolation_forest.predict(scaled_data)
isolation_outliers = np.array([1 if label == -1 else 0 for label in isolation_outliers])

In [ ]:
anomalies_report(isolation_outliers)

In [ ]:
labeled_data = data_features.copy()
labeled_data['is_outlier'] = isolation_outliers

sns.pairplot(data=labeled_data, vars = other_features,
             hue='is_outlier', hue_order=[1, 0],
             markers=['x', 'o'],  palette='bright')


Плюсы и минусы
+ Снова нелинейные разделяющие границы, интуитивно понятные принципы работы

+ Снова удобно использовать, когда не можем в бинарную классификацию

- Довольно сложная настройка параметров, которую тяжело проводить, не имея хотя бы минимальной разметки или предположения о количестве "грязных" наблюдений - параметр contamination

##Финальное сравнение¶

In [ ]:
summary = np.concatenate((
    [std_outliers],
    [iqr_outliers],
    [euclidian_outliers],
    [cityblock_outliers],
    [density_outlier],
    [svm_outliers],
    [isolation_outliers]
))

In [ ]:

summary = pd.DataFrame(
    summary.T,
    columns=['std', 'iqr', 'euclid', 'cityblock', 'dbscan', 'svm', 'isolation']
)
summary.head()

In [ ]:
summary.sum(axis=1).value_counts()

In [ ]:
outlier_score = summary.mean(axis=1)
plt.hist(outlier_score, alpha=0.6);

##All combined

In [ ]:
simple_score = outlier_score.apply(lambda x: 0 if x < 0.4 else 0.5 if x < 0.8 else 1)

labeled_data = data_features.copy()
labeled_data['outlier_score'] = simple_score

custom_palette = {0:'g', 0.5:'b', 1.0:'r'}

sns.pairplot(data=labeled_data, vars=other_features,
             hue='outlier_score',
             hue_order=[1, 0.5, 0],
             palette=custom_palette
)

##Just model based

In [ ]:

outliers_score_model_based = summary[['dbscan', 'svm', 'isolation']].sum(axis=1)
plt.hist(outliers_score_model_based, alpha=0.6);
outliers_score_model_based.value_counts()

In [ ]:
labeled_data = data_features.copy()
labeled_data['outlier_score'] = outliers_score_model_based

custom_palette = {0:'g', 1:'b', 2:'y', 3:'r'}

sns.pairplot(data=labeled_data, vars=other_features,
             hue='outlier_score',
             hue_order=[3, 2, 1, 0],
             palette=custom_palette
)

In [ ]:
sns.pairplot(data=labeled_data[labeled_data.outlier_score!=3],
             vars=other_features,
             hue='outlier_score',
             hue_order=[3, 2, 1, 0],
             palette=custom_palette
)

In [ ]:
sns.pairplot(data=labeled_data[labeled_data.outlier_score.isin([0, 1])],
             vars=other_features,
             hue='outlier_score',
             hue_order=[3, 2, 1, 0],
             palette=custom_palette
)

In [ ]:
sns.pairplot(data=labeled_data[labeled_data.outlier_score.isin([0])],
             vars=other_features,
             hue='outlier_score',
             hue_order=[3, 2, 1, 0],
             palette=custom_palette
)


Дополнительные материалы

Поиск аномалий во временных рядах https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-9-time-series-analysis-in-python-a270cb05e0b3
Плотностные методы для поиска аномалий https://towardsdatascience.com/density-based-algorithm-for-outlier-detection-8f278d2f7983
Верхнеуровневая библиотека с кучей методов https://www.analyticsvidhya.com/blog/2019/02/outlier-detection-python-pyod/
Еще раз про классику https://blog.floydhub.com/introduction-to-anomaly-detection-in-python/